In [2]:
import open3d as o3d
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d
import laspy
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Loading .LAS file

In [3]:
cfg_file = "/home/jeevin/Open3D-ML/ml3d/configs/randlanet_semantickitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)


model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "/home/jeevin/Open3D-ML/jeevin/BLOK_D_1.las"
dataset = ml3d.datasets.SemanticKITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)


# download the weights.
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = ckpt_folder + "randlanet_semantickitti_202201071330utc.pth"
randlanet_url = "https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth"
if not os.path.exists(ckpt_path):
    cmd = "wget {} -O {}".format(randlanet_url, ckpt_path)
    os.system(cmd)

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("test")
data = test_split.get_data(0)

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()

--2024-06-21 00:21:35--  https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.199.251, 216.58.200.27, 142.251.223.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.199.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5101179 (4.9M) [application/octet-stream]
Saving to: ‘./logs/randlanet_semantickitti_202201071330utc.pth’

     0K .......... .......... .......... .......... ..........  1% 3.23M 1s
    50K .......... .......... .......... .......... ..........  2% 7.23M 1s
   100K .......... .......... .......... .......... ..........  3% 10.8M 1s
   150K .......... .......... .......... .......... ..........  4% 14.4M 1s
   200K .......... .......... .......... .......... ..........  5% 21.2M 1s
   250K .......... .......... .......... .......... ..........  6% 23.4M 1s
   300K .......... .......... .......... .......

NotADirectoryError: [Errno 20] Not a directory: '/home/jeevin/Open3D-ML/jeevin/BLOK_D_1.las/dataset/sequences/11/velodyne'

In [2]:
las_file_path = "BLOK D 1.las"
abs_path = os.path.abspath(las_file_path)
print(abs_path)
las = laspy.read(las_file_path)

# Convert to numpy
points = np.vstack((las.x, las.y, las.z)).transpose()

/home/jeevin/Open3D-ML/jeevin/BLOK D 1.las


Create an Open3D PointCloud object

In [5]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

if hasattr(las, 'red') and hasattr(las, 'green') and hasattr(las, 'blue'):
    colors = np.vstack((las.red, las.green, las.blue)).transpose()
    colors = colors / 65535.0  # Normalize the color values if necessary
    pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])